<h1>CycleGAN for Style Transfer</h1>
<h3> Imports </h3>

In [21]:
#Python imports
import torch
import torch.nn as tnn
from torch.autograd import Variable
import sys
import itertools
from torch.utils.data import DataLoader
from Dataset import ImageSet
import datetime
import torchvision.utils as tv
import os
import matplotlib.pyplot as plt
import numpy as np

#Import network elements
import network_elements as net

#stdout code blocks reload's print statement
#prevstd = sys.stdout
#sys.stdout = None
reload(net)
print("Reloading net")
#sys.stdout = prevstd


Reloading net


<h3>Set parameters</h3>

In [22]:
#The number of channels in the input image
im_in_channels = 3

#The number of channels in the output image
im_out_channels = 3

#The size of the largest size of the input image
im_size = 256

LEARNING_RATE = 0.0002

LR_DECAY_START = 100
LR_DECAY_END = 200
NUM_EPOCHS = 200

<h3>Import dataset</h3>

In [23]:
imageSet = ImageSet();
dataset = "horse2zebra"
imageSet.loadData(dataset, 'train', im_size)
imgLoader = DataLoader(imageSet,shuffle=True)

./datasets/horse2zebra
Finished loading data


<h3>Define the CycleGAN network</h3>

In [24]:
#G : X->Y
G = net.Mapping(im_in_channels, im_out_channels, im_size)

#F : Y->X
F = net.Mapping(im_out_channels, im_out_channels, im_size)

#G mapping discriminator
Dx = net.Discriminator(im_in_channels)

#F mapping discriminator
Dy = net.Discriminator(im_out_channels)

#Initialize CUDA implementations
G.cuda();
F.cuda();
Dx.cuda();
Dy.cuda();

#Define losses
cycle_loss = tnn.L1Loss(); #Cycle-consistency loss
gan_loss = tnn.MSELoss(); #Adversarial loss
identity_loss = tnn.L1Loss(); #Prevents image tinting

<h3>Initialize weights for each network</h3>

In [25]:
# norm_weights function applies Gaussian norm weights with mean 0 and stddev 0.02
G.apply(net.conv_norm_weights);
F.apply(net.conv_norm_weights);
Dx.apply(net.conv_norm_weights);
Dy.apply(net.conv_norm_weights);

<h3>Set optimizer parameters</h3>

In [26]:
#simplified version has no pooling, paper does not, paper code does...
# REMOVE

gan_opt = torch.optim.Adam(itertools.chain(G.parameters(), F.parameters()),
                           lr = LEARNING_RATE)
dx_opt = torch.optim.Adam(Dx.parameters(), lr = LEARNING_RATE)
dy_opt = torch.optim.Adam(Dy.parameters(), lr = LEARNING_RATE)

gan_lr = torch.optim.lr_scheduler.LambdaLR(gan_opt, net.LRDecay(LR_DECAY_START, LR_DECAY_END).step)
dx_lr = torch.optim.lr_scheduler.LambdaLR(dx_opt, net.LRDecay(LR_DECAY_START, LR_DECAY_END).step)
dy_lr = torch.optim.lr_scheduler.LambdaLR(dy_opt, net.LRDecay(LR_DECAY_START, LR_DECAY_END).step)

The paper uses Adam optimizer with a learning rate of 0.0002 for the first 100 epochs that linearly decays to 0 over the next 100 epochs.

<h3> Initialize training variables </h3>

In [27]:
x = torch.cuda.FloatTensor(1, im_in_channels, im_size, im_size)
y = torch.cuda.FloatTensor(1, im_out_channels, im_size, im_size)

# Variable wrapper is depricated in >0.4
dReal = Variable(torch.cuda.FloatTensor(1).fill_(1.0), requires_grad = False)
dFake = Variable(torch.cuda.FloatTensor(1).fill_(0.0), requires_grad = False)

buffX_Y = net.MapBuffer()
buffY_X = net.MapBuffer()

if not os.path.isdir("./img/"):
    os.mkdir("./img/")
    
if not os.path.isdir("./log/"):
    os.mkdir("./log/")
    
if os.path.isfile("./log/loss.log"):    
    os.remove("./log/loss.log")
    
logfile = open("./log/loss.log","a")

<h3> Training </h3>

In [ ]:
start_time = datetime.datetime.now()
for i in range(0,NUM_EPOCHS):
    for j, img in enumerate(imgLoader):
        elapsed_time = datetime.datetime.now()-start_time
        print("Epoch %d/%d, %d/%d, %s"%(i,LR_DECAY_END,j,len(imageSet),str(elapsed_time)))
        src_x = Variable(x.copy_(img['x']))
        src_y = Variable(y.copy_(img['y']))
        
        gan_opt.zero_grad()
        
        #identity Loss
        mapY_Y = G(src_y)
        lossY_Y = identity_loss(mapY_Y, src_y)*5.0
        
        mapX_X = F(src_x)
        lossX_X = identity_loss(mapX_X, src_x)*5.0
        
        #GAN Loss
        mapX_Y = G(src_x)
        dX_Y = Dx(mapX_Y)
        lossX_Y = gan_loss(dX_Y, dReal)
        
        mapY_X = F(src_y)
        dY_X = Dy(mapY_X)
        lossY_X = gan_loss(dY_X, dReal)
        
        #Cycle Consistency Loss
        mapX_Y_X = F(mapX_Y)
        lossX_Y_X = cycle_loss(mapX_Y_X, src_x)*10.0
        
        mapY_X_Y = G(mapY_X)
        lossY_X_Y = cycle_loss(mapY_X_Y, src_y)*10.0
        
        mapLoss = lossY_Y + lossX_X + lossX_Y + lossY_X + lossX_Y_X + lossY_X_Y
        mapLoss.backward()
        
        gan_opt.step()
        
        #Dx
        dx_opt.zero_grad()
        
        dX_res = Dx(src_x)
        lossX = gan_loss(dX_res, dReal)
        
        mapY_X = buffY_X.cycle(mapY_X)
        dY_X = Dx(mapY_X.detach())
        lossY_X = gan_loss(dY_X, dFake)
        
        dxLoss = (lossX + lossY_X)*0.5
        dxLoss.backward()
        
        dx_opt.step()
        
        #Dy
        dy_opt.zero_grad()
        
        dY_res = Dy(src_y)
        lossY = gan_loss(dY_res, dReal)
        
        mapX_Y = buffX_Y.cycle(mapX_Y)
        dX_Y = Dy(mapX_Y.detach())
        lossX_Y = gan_loss(dX_Y, dFake)
        
        dyLoss = (lossY + lossX_Y)*0.5
        dyLoss.backward()
        
        dy_opt.step()     
        
        
        
        logfile.write("%d,%d,%f,%f,%f,%f,%f\n"%(i,j,mapLoss.data[0], 
                                          (lossX_X+lossY_Y).data[0], 
                                          (lossX_Y+lossY_X).data[0], 
                                          (lossX_Y_X+lossY_X_Y).data[0], 
                                          (dxLoss+dyLoss).data[0]))
        
    plt.imshow(np.transpose(torch.squeeze(mapX_Y.data.cpu()).numpy(), (1,2,0)), interpolation='nearest')
    tv.save_image(src_x.data, './img/src_x_%d.jpg'%(i))
    tv.save_image(src_y.data, './img/src_x_%d.jpg'%(i))
    tv.save_image(mapX_Y.data, './img/mapX_Y_%d.jpg'%(i))
    tv.save_image(mapY_X.data, './img/mapY_X_%d.jpg'%(i))
    gan_lr.step()
    dx_lr.step()
    dy_lr.step()
    

    if not os.path.isdir("./model/"):
        os.mkdir("./model/")
    model_file = open("./model/model_info.txt", "w")
    model_file.write("%d"%i)
    model_file.close()
    torch.save(G.state_dict(), './model/G.data')
    torch.save(F.state_dict(), './model/F.data')
    torch.save(Dx.state_dict(), './model/Dx.data')
    torch.save(Dy.state_dict(), './model/Dy.data')

Epoch 0/200, 0/1334, 0:00:00.013294
Epoch 0/200, 1/1334, 0:00:00.629329
Epoch 0/200, 2/1334, 0:00:01.248054
Epoch 0/200, 3/1334, 0:00:01.816241
Epoch 0/200, 4/1334, 0:00:02.424329
Epoch 0/200, 5/1334, 0:00:03.035997
Epoch 0/200, 6/1334, 0:00:03.620506
Epoch 0/200, 7/1334, 0:00:04.217504
Epoch 0/200, 8/1334, 0:00:04.818127
Epoch 0/200, 9/1334, 0:00:05.432754
Epoch 0/200, 10/1334, 0:00:06.011560
Epoch 0/200, 11/1334, 0:00:06.617616
Epoch 0/200, 12/1334, 0:00:07.202756
Epoch 0/200, 13/1334, 0:00:07.813974
Epoch 0/200, 14/1334, 0:00:08.417332
Epoch 0/200, 15/1334, 0:00:09.012622
Epoch 0/200, 16/1334, 0:00:09.610414
Epoch 0/200, 17/1334, 0:00:10.219612
Epoch 0/200, 18/1334, 0:00:10.815230
Epoch 0/200, 19/1334, 0:00:11.415040
Epoch 0/200, 20/1334, 0:00:12.013251
Epoch 0/200, 21/1334, 0:00:12.610934
Epoch 0/200, 22/1334, 0:00:13.215738
Epoch 0/200, 23/1334, 0:00:13.819925
Epoch 0/200, 24/1334, 0:00:14.412275
Epoch 0/200, 25/1334, 0:00:15.020535
Epoch 0/200, 26/1334, 0:00:15.622082
Epoch 0/200

Epoch 0/200, 219/1334, 0:02:11.422333
Epoch 0/200, 220/1334, 0:02:12.028177
Epoch 0/200, 221/1334, 0:02:12.617236
Epoch 0/200, 222/1334, 0:02:13.226964
Epoch 0/200, 223/1334, 0:02:13.822599
Epoch 0/200, 224/1334, 0:02:14.417572
Epoch 0/200, 225/1334, 0:02:15.023480
Epoch 0/200, 226/1334, 0:02:15.622449
Epoch 0/200, 227/1334, 0:02:16.214860
Epoch 0/200, 228/1334, 0:02:16.817189
Epoch 0/200, 229/1334, 0:02:17.418487
Epoch 0/200, 230/1334, 0:02:18.020523
Epoch 0/200, 231/1334, 0:02:18.615031
Epoch 0/200, 232/1334, 0:02:19.216115
Epoch 0/200, 233/1334, 0:02:19.820449
Epoch 0/200, 234/1334, 0:02:20.425439
Epoch 0/200, 235/1334, 0:02:21.009498
Epoch 0/200, 236/1334, 0:02:21.617840
Epoch 0/200, 237/1334, 0:02:22.218236
Epoch 0/200, 238/1334, 0:02:22.814515
Epoch 0/200, 239/1334, 0:02:23.423036
Epoch 0/200, 240/1334, 0:02:24.023790
Epoch 0/200, 241/1334, 0:02:24.628005
Epoch 0/200, 242/1334, 0:02:25.223039
Epoch 0/200, 243/1334, 0:02:25.814608
Epoch 0/200, 244/1334, 0:02:26.425445
Epoch 0/200,

Epoch 0/200, 435/1334, 0:04:21.008452
Epoch 0/200, 436/1334, 0:04:21.612586
Epoch 0/200, 437/1334, 0:04:22.222978
Epoch 0/200, 438/1334, 0:04:22.810701
Epoch 0/200, 439/1334, 0:04:23.413979
Epoch 0/200, 440/1334, 0:04:24.003817
Epoch 0/200, 441/1334, 0:04:24.613750
Epoch 0/200, 442/1334, 0:04:25.208298
Epoch 0/200, 443/1334, 0:04:25.809586
Epoch 0/200, 444/1334, 0:04:26.421908
Epoch 0/200, 445/1334, 0:04:27.011491
Epoch 0/200, 446/1334, 0:04:27.617296
Epoch 0/200, 447/1334, 0:04:28.201641
Epoch 0/200, 448/1334, 0:04:28.812850
Epoch 0/200, 449/1334, 0:04:29.409905
Epoch 0/200, 450/1334, 0:04:30.012208
Epoch 0/200, 451/1334, 0:04:30.611574
Epoch 0/200, 452/1334, 0:04:31.213446
Epoch 0/200, 453/1334, 0:04:31.807848
Epoch 0/200, 454/1334, 0:04:32.405195
Epoch 0/200, 455/1334, 0:04:33.016017
Epoch 0/200, 456/1334, 0:04:33.611874
Epoch 0/200, 457/1334, 0:04:34.220617
Epoch 0/200, 458/1334, 0:04:34.814930
Epoch 0/200, 459/1334, 0:04:35.411080
Epoch 0/200, 460/1334, 0:04:36.015450
Epoch 0/200,

Epoch 0/200, 651/1334, 0:06:30.631445
Epoch 0/200, 652/1334, 0:06:31.225224
Epoch 0/200, 653/1334, 0:06:31.822786
Epoch 0/200, 654/1334, 0:06:32.426678
Epoch 0/200, 655/1334, 0:06:33.021526
Epoch 0/200, 656/1334, 0:06:33.613001
Epoch 0/200, 657/1334, 0:06:34.215561
Epoch 0/200, 658/1334, 0:06:34.870125
Epoch 0/200, 659/1334, 0:06:35.430048
Epoch 0/200, 660/1334, 0:06:36.023795
Epoch 0/200, 661/1334, 0:06:36.617580
Epoch 0/200, 662/1334, 0:06:37.219754
Epoch 0/200, 663/1334, 0:06:37.816925
Epoch 0/200, 664/1334, 0:06:38.418145
Epoch 0/200, 665/1334, 0:06:39.017331
Epoch 0/200, 666/1334, 0:06:39.627751
Epoch 0/200, 667/1334, 0:06:40.229695
Epoch 0/200, 668/1334, 0:06:40.820484
Epoch 0/200, 669/1334, 0:06:41.429294
Epoch 0/200, 670/1334, 0:06:42.020594
Epoch 0/200, 671/1334, 0:06:42.617617
Epoch 0/200, 672/1334, 0:06:43.235538
Epoch 0/200, 673/1334, 0:06:43.832179
Epoch 0/200, 674/1334, 0:06:44.433306
Epoch 0/200, 675/1334, 0:06:45.026873
Epoch 0/200, 676/1334, 0:06:45.624274
Epoch 0/200,

Epoch 0/200, 867/1334, 0:08:40.220059
Epoch 0/200, 868/1334, 0:08:40.825279
Epoch 0/200, 869/1334, 0:08:41.417292
Epoch 0/200, 870/1334, 0:08:42.018506
Epoch 0/200, 871/1334, 0:08:42.618760
Epoch 0/200, 872/1334, 0:08:43.221766
Epoch 0/200, 873/1334, 0:08:43.821262
Epoch 0/200, 874/1334, 0:08:44.426147
Epoch 0/200, 875/1334, 0:08:45.022653
Epoch 0/200, 876/1334, 0:08:45.618425
Epoch 0/200, 877/1334, 0:08:46.217186
Epoch 0/200, 878/1334, 0:08:46.819617
Epoch 0/200, 879/1334, 0:08:47.419972
Epoch 0/200, 880/1334, 0:08:48.018119
Epoch 0/200, 881/1334, 0:08:48.626697
Epoch 0/200, 882/1334, 0:08:49.223091
Epoch 0/200, 883/1334, 0:08:49.822988
Epoch 0/200, 884/1334, 0:08:50.419130
Epoch 0/200, 885/1334, 0:08:51.023492
Epoch 0/200, 886/1334, 0:08:51.628371
Epoch 0/200, 887/1334, 0:08:52.221111
Epoch 0/200, 888/1334, 0:08:52.815518
Epoch 0/200, 889/1334, 0:08:53.423084
Epoch 0/200, 890/1334, 0:08:54.021893
Epoch 0/200, 891/1334, 0:08:54.617052
Epoch 0/200, 892/1334, 0:08:55.228172
Epoch 0/200,

...

Epoch 10/200, 0/1334, 2:13:32.475311
Epoch 10/200, 1/1334, 2:13:33.033384
Epoch 10/200, 2/1334, 2:13:33.619092
Epoch 10/200, 3/1334, 2:13:34.221076
Epoch 10/200, 4/1334, 2:13:34.870147
Epoch 10/200, 5/1334, 2:13:35.428810
Epoch 10/200, 6/1334, 2:13:36.026715
Epoch 10/200, 7/1334, 2:13:36.622310
Epoch 10/200, 8/1334, 2:13:37.207656
Epoch 10/200, 9/1334, 2:13:37.815689
Epoch 10/200, 10/1334, 2:13:38.414972
Epoch 10/200, 11/1334, 2:13:39.028709
Epoch 10/200, 12/1334, 2:13:39.634088
Epoch 10/200, 13/1334, 2:13:40.217121
Epoch 10/200, 14/1334, 2:13:40.819704
Epoch 10/200, 15/1334, 2:13:41.416434
Epoch 10/200, 16/1334, 2:13:42.019699
Epoch 10/200, 17/1334, 2:13:42.617704
Epoch 10/200, 18/1334, 2:13:43.223267
Epoch 10/200, 19/1334, 2:13:43.821926
Epoch 10/200, 20/1334, 2:13:44.405825
Epoch 10/200, 21/1334, 2:13:45.008193
Epoch 10/200, 22/1334, 2:13:45.609613
Epoch 10/200, 23/1334, 2:13:46.218919
Epoch 10/200, 24/1334, 2:13:46.807355
Epoch 10/200, 25/1334, 2:13:47.420943
Epoch 10/200, 26/1334,

Epoch 10/200, 213/1334, 2:15:40.316993
Epoch 10/200, 214/1334, 2:15:40.918754
Epoch 10/200, 215/1334, 2:15:41.524922
Epoch 10/200, 216/1334, 2:15:42.133411
Epoch 10/200, 217/1334, 2:15:42.723502
Epoch 10/200, 218/1334, 2:15:43.315453
Epoch 10/200, 219/1334, 2:15:43.936647
Epoch 10/200, 220/1334, 2:15:44.524859
Epoch 10/200, 221/1334, 2:15:45.119423
Epoch 10/200, 222/1334, 2:15:45.718417
Epoch 10/200, 223/1334, 2:15:46.318819
Epoch 10/200, 224/1334, 2:15:46.909051
Epoch 10/200, 225/1334, 2:15:47.517087
Epoch 10/200, 226/1334, 2:15:48.117537
Epoch 10/200, 227/1334, 2:15:48.717602
Epoch 10/200, 228/1334, 2:15:49.323999
Epoch 10/200, 229/1334, 2:15:49.917100
Epoch 10/200, 230/1334, 2:15:50.522074
Epoch 10/200, 231/1334, 2:15:51.114566
Epoch 10/200, 232/1334, 2:15:51.724585
Epoch 10/200, 233/1334, 2:15:52.325905
Epoch 10/200, 234/1334, 2:15:52.917959
Epoch 10/200, 235/1334, 2:15:53.520559
Epoch 10/200, 236/1334, 2:15:54.118680
Epoch 10/200, 237/1334, 2:15:54.723442
Epoch 10/200, 238/1334, 2

Epoch 10/200, 424/1334, 2:17:46.918490
Epoch 10/200, 425/1334, 2:17:47.516478
Epoch 10/200, 426/1334, 2:17:48.122227
Epoch 10/200, 427/1334, 2:17:48.719242
Epoch 10/200, 428/1334, 2:17:49.318897
Epoch 10/200, 429/1334, 2:17:49.917225
Epoch 10/200, 430/1334, 2:17:50.520351
Epoch 10/200, 431/1334, 2:17:51.114435
Epoch 10/200, 432/1334, 2:17:51.713726
Epoch 10/200, 433/1334, 2:17:52.311773
Epoch 10/200, 434/1334, 2:17:52.912894
Epoch 10/200, 435/1334, 2:17:53.508340
Epoch 10/200, 436/1334, 2:17:54.128381
Epoch 10/200, 437/1334, 2:17:54.730207
Epoch 10/200, 438/1334, 2:17:55.319135
Epoch 10/200, 439/1334, 2:17:55.911709
Epoch 10/200, 440/1334, 2:17:56.517839
Epoch 10/200, 441/1334, 2:17:57.120600
Epoch 10/200, 442/1334, 2:17:57.717720
Epoch 10/200, 443/1334, 2:17:58.320231
Epoch 10/200, 444/1334, 2:17:58.925517
Epoch 10/200, 445/1334, 2:17:59.517299
Epoch 10/200, 446/1334, 2:18:00.119380
Epoch 10/200, 447/1334, 2:18:00.718992
Epoch 10/200, 448/1334, 2:18:01.319414
Epoch 10/200, 449/1334, 2

Epoch 10/200, 635/1334, 2:19:53.521844
Epoch 10/200, 636/1334, 2:19:54.117262
Epoch 10/200, 637/1334, 2:19:54.721086
Epoch 10/200, 638/1334, 2:19:55.319234
Epoch 10/200, 639/1334, 2:19:55.925758
Epoch 10/200, 640/1334, 2:19:56.529689
Epoch 10/200, 641/1334, 2:19:57.125730
Epoch 10/200, 642/1334, 2:19:57.715256
Epoch 10/200, 643/1334, 2:19:58.319263
Epoch 10/200, 644/1334, 2:19:58.920820
Epoch 10/200, 645/1334, 2:19:59.520000
Epoch 10/200, 646/1334, 2:20:00.117109
Epoch 10/200, 647/1334, 2:20:00.721578
Epoch 10/200, 648/1334, 2:20:01.323850
Epoch 10/200, 649/1334, 2:20:01.920955
Epoch 10/200, 650/1334, 2:20:02.519871
Epoch 10/200, 651/1334, 2:20:03.121377
Epoch 10/200, 652/1334, 2:20:03.717960
Epoch 10/200, 653/1334, 2:20:04.371037
Epoch 10/200, 654/1334, 2:20:04.931839
Epoch 10/200, 655/1334, 2:20:05.519949
Epoch 10/200, 656/1334, 2:20:06.125176
Epoch 10/200, 657/1334, 2:20:06.724593
Epoch 10/200, 658/1334, 2:20:07.322230
Epoch 10/200, 659/1334, 2:20:07.918254
Epoch 10/200, 660/1334, 2

Epoch 10/200, 846/1334, 2:22:00.122438
Epoch 10/200, 847/1334, 2:22:00.736687
Epoch 10/200, 848/1334, 2:22:01.329503
Epoch 10/200, 849/1334, 2:22:01.935612
Epoch 10/200, 850/1334, 2:22:02.531722
Epoch 10/200, 851/1334, 2:22:03.121884
Epoch 10/200, 852/1334, 2:22:03.713187
Epoch 10/200, 853/1334, 2:22:04.320467
Epoch 10/200, 854/1334, 2:22:04.921628
Epoch 10/200, 855/1334, 2:22:05.522173
Epoch 10/200, 856/1334, 2:22:06.129712
Epoch 10/200, 857/1334, 2:22:06.717842
Epoch 10/200, 858/1334, 2:22:07.321418
Epoch 10/200, 859/1334, 2:22:07.924201
Epoch 10/200, 860/1334, 2:22:08.525231
Epoch 10/200, 861/1334, 2:22:09.118316
Epoch 10/200, 862/1334, 2:22:09.722343
Epoch 10/200, 863/1334, 2:22:10.325904
Epoch 10/200, 864/1334, 2:22:10.917578
Epoch 10/200, 865/1334, 2:22:11.522958
Epoch 10/200, 866/1334, 2:22:12.174920
Epoch 10/200, 867/1334, 2:22:12.733928
Epoch 10/200, 868/1334, 2:22:13.329277
Epoch 10/200, 869/1334, 2:22:13.920641
Epoch 10/200, 870/1334, 2:22:14.514058
Epoch 10/200, 871/1334, 2

In [ ]:
close(logfile)

In [ ]:
model_file = open("./model/model_info.txt", "w")
model_file.write("%d"%i)
model_file.close()  